In [3]:
import os.path
import pickle
from utility import load_categories

from utility import cp_to_date,load_appliance_time, time_lst_to_dic, merge_overlapping_intervals, plot_activity, load_daily_activity_dic,load_activity_dic
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import os
plt.rcParams['font.family'] = 'Times New Roman'
import numpy as np
from datetime import timedelta
warnings.filterwarnings("ignore")

In [4]:
def merge_overlapping_intervals(appliance_time, tolerance):
    # 将时间对转换为pandas的Timestamp对象并按开始时间排序
    appliance_time = sorted([(pd.Timestamp(start), pd.Timestamp(end)) for start, end in appliance_time], key=lambda x: x[0])

    # 合并重叠的时间段
    merged_intervals = []
    for start, end in appliance_time:
        # 如果列表为空或当前时间段的开始时间大于合并列表中最后一个时间段的结束时间/两事件发生间隔小于大于5分钟，则添加新的时间段
        if merged_intervals==[]:
            merged_intervals.append([start, end])
        elif start-merged_intervals[-1][1]>timedelta(minutes=tolerance):
            merged_intervals.append([start, end])
        else:
            # 否则，更新合并列表中最后一个时间段的结束时间为当前时间段的结束时间
            merged_intervals[-1][1] = max(merged_intervals[-1][1], end)

    # 将合并后的时间段转换为元组并返回
    return [(start, end) for start, end in merged_intervals]

def appiance_activity_association(cp_path, appliance_path, appliance_lst, tolerance):
    # load changepoints
    with open(cp_path,'rb') as f:
        all_cp = pickle.load(f)
    cp_time = cp_to_date(all_cp,"2013-05-01 00:00:00",365)
    cp_time = [(cp_time[i], cp_time[i+1]) for i in range(len(cp_time)-1)]

    # process appliance
    appliance_time = load_appliance_time(appliance_path,appliance_lst)
    temporary_lst = [] # buffer
    for _, item in appliance_time.items():
        temporary_lst += item
    appliance_time = temporary_lst
    appliance_time = sorted(appliance_time, key=lambda x: x[0])
    appliance_time = merge_overlapping_intervals(appliance_time,tolerance)
    activity = find_activities(appliance_time, cp_time)
    return activity

def find_activities(cooking_time, meter_changes):
    activity = []
    # 遍历每一个变点监测时间段
    for start_meter, end_meter in meter_changes:
        min_start = None
        max_end = None

        # 检查cooking_time中是否有时间段与当前变点监测时间段重叠
        for start_cook, end_cook in cooking_time:
            if start_meter< start_cook < end_meter:
                # 找到重叠时间段中的最早开始时间和最晚结束时间
                if min_start is None or start_cook < min_start:
                    min_start = start_cook
                if max_end is None or end_cook > max_end:
                    max_end = end_cook
                cooking_time.remove((start_cook,end_cook))

        # 如果存在重叠时间段，则将其添加到activity列表中
        if min_start is not None and max_end is not None:
            activity.append((min_start, max_end))

    return activity
acitivity_path = r'C:\Users\ASUS\Desktop\Few-shot NILM (2)\activity_annotation\time_data\cleaning'
cp_path = r"change_points_30s_120.pkl"
entertainment = appiance_activity_association(cp_path, acitivity_path, ['washing_machine','dishwasher',
                                                    'hair_dryer','straighteners',"iron",'vacuum'],30)

In [ ]:
file_path=r'C:\annoticity-master (2)\anno\datasets\UK_DALE\ukdale.h5'
# cleaning_df = load_categories(file_path, [22], ["hoover"], 'active')
kitchen_df = load_categories(file_path, [10,11,13,16,42], ["kettle", "toaster",'microwave',"breadmaker","gas oven"], 'active')
entertainment_df = load_categories(file_path, [7,9,17,28],["tv", "htpc","amp_livingroom",' subwoofer_livingroom'],'active')
cleaning_df = load_categories(file_path, [5,6,39,40,41,22],['washing machine','dishwasher',
                                                    'hair dryer','straighteners',"iron",'vacuum'],'active')
aggregate_df = load_categories(file_path, [1],["aggregated"], 'apparent')
office_df = load_categories(file_path, [4,51,53],['laptop','office_pc','printer'],'active')

In [5]:
activity_path = r'C:\Users\ASUS\Desktop\Few-shot NILM (2)\activity_annotation\activity_120'
activity =load_daily_activity_dic(activity_path, ['cooking','cleaning','entertainment',"working"])

In [6]:
# import pickle
# with open(r'C:\Users\ASUS\Desktop\Few-shot NILM (2)\activity_annotation\activity_120\cleaning.pkl', 'wb') as f:
#     pickle.dump(entertainment, f)

In [7]:
import numpy as np
with open("change_points_30s_120.pkl",'rb') as f:
    cp = pickle.load(f)

In [ ]:
start = pd.to_datetime("2013-10-24 00:00:00")
end = pd.to_datetime("2013-10-25 00:00:00")
time_difference = start - pd.to_datetime("2013-05-01 00:00:00")

In [ ]:
time_difference.days

In [ ]:
aggregate_df.index

In [ ]:
plt.plot(aggregate_df[start:end], label="aggregate")
for i in np.array(cp[176])-10:
    plt.axvline(aggregate_df[start:end].index[i].tz_convert('Europe/London'), color="red")
plt.savefig('cp_detection.jpg')


In [ ]:
start = pd.to_datetime("2013-10-26 00:00:00")
end = pd.to_datetime("2013-10-27 00:00:00")
plt.figure(figsize=(18, 12))
plt.subplot(5, 1, 1)
plt.plot(aggregate_df[start:end], label="aggregate")
for i in np.array(cp[176])-10:
    plt.axvline(aggregate_df[start:end].index[i].tz_convert('Europe/London'), color="red")
####################
plt.subplot(5, 1, 2)
for i in kitchen_df.columns:
    plt.plot(kitchen_df[i][start:end].index, kitchen_df[i][start:end].values, label=i)
for start_time, end_time in activity['cooking'][start.date()]:
    plt.axvline(pd.to_datetime(start_time).tz_localize('Europe/London'), color="red")
    plt.axvline(pd.to_datetime(end_time).tz_localize('Europe/London'), color="red")
plt.legend()

#####################
plt.subplot(5, 1, 3)
for i in cleaning_df.columns:
    plt.plot(cleaning_df[i][start:end].index, cleaning_df[i][start:end].values, label=i)
for start_time, end_time in activity['cleaning'][start.date()]:
    plt.axvline(pd.to_datetime(start_time).tz_localize('Europe/London'), color="red")
    plt.axvline(pd.to_datetime(end_time).tz_localize('Europe/London'), color="red")
plt.legend()


############
plt.subplot(5, 1, 4)
for i in entertainment_df.columns:
    plt.plot(entertainment_df[i][start:end].index, entertainment_df[i][start:end].values, label=i)
for start_time, end_time in activity['entertainment'][start.date()]:
    plt.axvline(pd.to_datetime(start_time).tz_localize('Europe/London'), color="red")
    plt.axvline(pd.to_datetime(end_time).tz_localize('Europe/London'), color="red")
    plt.grid("--")
plt.legend()

##########
plt.subplot(5, 1, 5)
for i in office_df.columns:
    plt.plot(office_df[i][start:end].index, office_df[i][start:end].values, label=i)

for start_time, end_time in activity['working'][start.date()]:
    plt.axvline(pd.to_datetime(start_time).tz_localize('Europe/London'), color="red")
    plt.axvline(pd.to_datetime(end_time).tz_localize('Europe/London'), color="red")

# for i in kitchen_df.columns:
#     plt.plot(kitchen_df[i][start:end].index, kitchen_df[i][start:end].values, label=i)
# for start_time, end_time in cooking[start.date()]:
#     plt.axvline(pd.to_datetime(start_time).tz_localize('Europe/London'), color="red")
#     plt.axvline(pd.to_datetime(end_time).tz_localize('Europe/London'), color="red")
plt.legend()
plt.tight_layout()
plt.savefig('activity.png')
plt.savefig('activity.pdf')

# plt.show()

In [ ]:
office_df

In [ ]:
activity_path = r'C:\Users\ASUS\Desktop\Few-shot NILM (2)\activity_annotation\activity_120'

activity_dic = load_activity_dic(activity_path,["cooking",'cleaning',"entertainment",'working'])

In [ ]:
activity_dic["cooking"]

In [ ]:
# start = [start.time() for start, _ in activity if start<pd.Timestamp('2013-08-01T00:00:00')]
# end = [end.time() for _, end in activity if end<pd.Timestamp('2013-08-01T00:00:00')]

# plot activity duration

In [ ]:
statistic_cp = {
    "cooking": [600,1200,1320,1680,1920,2400],
    'cleaning':[820,1500,2100,2520],
    'entertainment':[500,1000,2100],
    'working':[820,1600,1900,2200]

}

In [ ]:

plt.figure(figsize=(10,5))
plt.rcParams.update({"font.size":12})
# from utility import plot_activity
activity_num = len(activity_dic.keys())
help_index = 1
for activity_name, time_index in activity_dic.items():
    plt.subplot(activity_num,1,help_index)
    start = [start.time() for start, _ in time_index if start<pd.Timestamp('2014-05-01T00:00:00')]
    end = [end.time() for _, end in time_index if end<pd.Timestamp('2014-05-01T00:00:00')]
    plot_activity(start,end)
    help_index+=1
    plt.ylabel(activity_name)
    activity_cp = statistic_cp[activity_name]
    for i in activity_cp:
        plt.axvline(i,color="red")
plt.tight_layout()

In [ ]:
def get_probability_activity(start,end):
    one_day_index = pd.date_range("00:00:00", "23:59:59", freq="30s")
    one_day_df = pd.DataFrame(index=pd.to_datetime(one_day_index), data=[0]*len(one_day_index))
    for start_time, end_time in zip(start,end):
        start_time =  one_day_df.index[0]+timedelta(hours=start_time.hour,minutes=start_time.minute,seconds=start_time.second)
        end_time = one_day_df.index[0]+timedelta(hours=end_time.hour,minutes=end_time.minute,seconds=end_time.second)
        one_day_df[str(start_time):str(end_time)]+=1
    return one_day_df.values
one_day_statistic = {}
for activity_name, time_index in activity_dic.items():
    start = [start.time() for start, _ in time_index if start<pd.Timestamp('2014-05-01T00:00:00')]
    end = [end.time() for _, end in time_index if end<pd.Timestamp('2014-05-01T00:00:00')]
    one_day_statistic[activity_name] = get_probability_activity(start,end)
    print(activity_name)


In [ ]:
activity_dic.keys()

In [ ]:
one_day_statistic.keys()

In [ ]:
plt.figure(figsize=(10,5))

plt.plot(one_day_statistic["cooking"])
plt.axvline([600])
plt.axvline([1200])
plt.axvline([1320])
plt.axvline([1680])
plt.axvline([2550])

# plot start,end

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
plt.figure(figsize=(10,5))


def plot_probability(start, end, freq="10min"):
    start_df = _retrieve_prob_df(start)
    end_df = _retrieve_prob_df(end)

    # 重采样并归一化
    start = start_df.resample(freq).mean()
    end = end_df.resample(freq).mean()
    start = start / start.sum()
    end = end / end.sum()

    # 找到时间索引的交集
    common_index = start.index.intersection(end.index)
    start_common = start.reindex(common_index, method='ffill').fillna(0)
    end_common = end.reindex(common_index, method='ffill').fillna(0)

    # 计算相交区域
    min_values = np.minimum(start_common.values, end_common.values)
    max_values = np.maximum(start_common.values, end_common.values)

    # 绘制底层区域（先绘制重叠区域下面的区域）
    plt.fill_between(start.index, start.values.reshape(-1), color='blue', alpha=0.5)
    plt.fill_between(end.index, end.values.reshape(-1), color='red', alpha=0.5)

    # 绘制相交区域
    plt.fill_between(common_index, min_values.reshape(-1), max_values.reshape(-1), color='purple', alpha=0.7)

    # 绘制时间序列
    plt.plot(start.index, start.values, color='blue')
    plt.plot(end.index, end.values, color='red')

    # 标签和网格
    plt.yticks([])
    plt.grid('--')
    plt.legend()

    point_index = [start.index[i] for i in range(0,len(start.index),len(start.index)//12)]+[start.index[-1]]
    time_index = [str(i) for i in range(0,24,2)]+[24]

    plt.xticks(point_index,time_index)


def _retrieve_prob_df(time):
    one_day_index = pd.date_range("00:00:00", "23:59:59", freq="30s")
    one_day_df = pd.DataFrame(index=pd.to_datetime(one_day_index), data=[0]*len(one_day_index))
    for i in time:
        date = datetime.combine(one_day_index[0], i)
        one_day_df.loc[str(date)] += 1
    return one_day_df
plot_probability(start,end)

In [ ]:

plt.figure(figsize=(10,5))
plt.rcParams.update({"font.size":12})
# from utility import plot_activity
activity_num = len(activity_dic.keys())
help_index = 1
for activity_name, time_index in activity_dic.items():
    plt.subplot(activity_num,1,help_index)
    start = [start.time() for start, _ in time_index if start<pd.Timestamp('2014-05-01T00:00:00')]
    end = [end.time() for _, end in time_index if end<pd.Timestamp('2014-05-01T00:00:00')]
    plot_probability(start,end)
    help_index+=1
    plt.ylabel(activity_name)

plt.tight_layout()

In [ ]:
# plot single activity start end

In [ ]:
activity = activity_dic["working"]
start = [start.time() for start, _ in activity if start<pd.Timestamp('2014-05-01T00:00:00')]
end = [end.time() for _, end in activity if end<pd.Timestamp('2014-05-01T00:00:00')]
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
plt.figure(figsize=(10,5))
def plot_probability(start, end, freq="10min"):
    start_df = _retrieve_prob_df(start)
    end_df = _retrieve_prob_df(end)

    # 重采样并归一化
    start = start_df.resample(freq).mean()
    end = end_df.resample(freq).mean()
    start = start / start.sum()
    end = end / end.sum()

    # 找到时间索引的交集
    common_index = start.index.intersection(end.index)
    start_common = start.reindex(common_index, method='ffill').fillna(0)
    end_common = end.reindex(common_index, method='ffill').fillna(0)

    # 计算相交区域的上下界
    min_values = np.minimum(start_common.values, end_common.values)
    max_values = np.maximum(start_common.values, end_common.values)

    # 找到相交区域的颜色
    color_intersection = np.where(start_common.values < end_common.values, 'blue', 'red')

    # 绘制相交区域
    for i in range(len(common_index)-1):
        plt.fill_between([common_index[i], common_index[i+1]],
                         min_values[i], max_values[i],
                         color=color_intersection[i], alpha=0.7)

    # 绘制底层区域
    plt.fill_between(start.index, start.values.reshape(-1), color='blue', alpha=0.5, label='Start')
    plt.fill_between(end.index, end.values.reshape(-1), color='red', alpha=0.5, label='End')

    # 绘制时间序列
    plt.plot(start.index, start.values.reshape(-1), color='blue')
    plt.plot(end.index, end.values.reshape(-1), color='red')

    # 标签和网格
    plt.yticks([])
    plt.grid('--')
    plt.legend()

    point_index = [start.index[i] for i in range(0,len(start.index),len(start.index)//12)]+[start.index[-1]]
    time_index = [str(i) for i in range(0,24,2)]+[24]

    plt.xticks(point_index,time_index)

    plt.show()

def _retrieve_prob_df(time):
    one_day_index = pd.date_range("00:00:00", "23:59:59", freq="30s")
    one_day_df = pd.DataFrame(index=pd.to_datetime(one_day_index), data=[0]*len(one_day_index))
    for i in time:
        date = datetime.combine(one_day_index[0], i)
        one_day_df.loc[str(date)] += 1
    return one_day_df
plot_probability(start,end)

In [ ]:
start